# Drafting

Pantograph supports drafting (technically the sketch step) from
[Draft-Sketch-Prove](https://github.com/wellecks/ntptutorial/tree/main/partII_dsp).
Pantograph's drafting feature is more powerful. At any place in the proof, you
can replace an expression with `sorry`, and the `sorry` will become a goal. Any type errors will also become goals. In order to detect whether type errors have occurred, the user can look at the messages from each compilation unit.

At this point we must introduce the idea of compilation units. Each Lean
definition, theorem, constant, etc., is a *compilation unit*. When Pantograph
extracts data from Lean source code, it sections the data into these compilation
units.

For example, consider this sketch produced by a language model prover:
```lean
by
   intros n m
   induction n with
   | zero =>
     have h_base: 0 + m = m := sorry
     have h_symm: m + 0 = m := sorry
     sorry
   | succ n ih =>
     have h_inductive: n + m = m + n := sorry
     have h_pull_succ_out_from_right: m + Nat.succ n = Nat.succ (m + n) := sorry
     have h_flip_n_plus_m: Nat.succ (n + m) = Nat.succ (m + n) := sorry
     have h_pull_succ_out_from_left: Nat.succ n + m = Nat.succ (n + m) := sorry
     sorry
```
There are some `sorry`s that we want to solve automatically with hammer tactics. We can do this by drafting. We first load the goal statement using `load_sorry`. We highly discourage writing more than 1 `sorry` in a theorem statement.

In [1]:
from pantograph import Server

sketch = """
theorem add_comm_proved_formal_sketch : ∀ n m : Nat, n + m = m + n := sorry
"""
server = await Server.create()
unit, = await server.load_sorry_async(sketch)
print(unit.goal_state)

payload: {"printDependentMVars": true}
payload: {"file": "\ntheorem add_comm_proved_formal_sketch : ∀ n m : Nat, n + m = m + n := sorry\n", "invocations": false, "sorrys": true, "newConstants": false, "readHeader": false, "inheritEnv": false, "typeErrorsAsGoals": false}

⊢ ∀ (n m : Nat), n + m = m + n


For an in-depth example, see `experiments/dsp`.

In [2]:
step = """
by
   -- Consider some n and m in Nats.
   intros n m
   -- Perform induction on n.
   induction n with
   | zero =>
     -- Base case: When n = 0, we need to show 0 + m = m + 0.
     -- We have the fact 0 + m = m by the definition of addition.
     have h_base: 0 + m = m := sorry
     -- We also have the fact m + 0 = m by the definition of addition.
     have h_symm: m + 0 = m := sorry
     -- Combine facts to close goal
     sorry
   | succ n ih =>
     sorry
"""
from pantograph.expr import TacticDraft
tactic = TacticDraft(step)
state1 = await server.goal_tactic_async(unit.goal_state, tactic)
print(state1)

payload: {"stateId": 0, "goalId": 0, "draft": "\nby\n   -- Consider some n and m in Nats.\n   intros n m\n   -- Perform induction on n.\n   induction n with\n   | zero =>\n     -- Base case: When n = 0, we need to show 0 + m = m + 0.\n     -- We have the fact 0 + m = m by the definition of addition.\n     have h_base: 0 + m = m := sorry\n     -- We also have the fact m + 0 = m by the definition of addition.\n     have h_symm: m + 0 = m := sorry\n     -- Combine facts to close goal\n     sorry\n   | succ n ih =>\n     sorry\n"}
n : Nat
m : Nat
⊢ 0 + m = m
n : Nat
m : Nat
h_base : 0 + m = m
⊢ m + 0 = m
n : Nat
m : Nat
h_base : 0 + m = m
h_symm : m + 0 = m
⊢ 0 + m = m + 0
n✝ : Nat
m : Nat
n : Nat
ih : n + m = m + n
⊢ n + 1 + m = m + (n + 1)
